# Cas hydro

In [1]:
import numpy as np
import multiprocessing
import numpy.linalg as la
import scipy.integrate as spi
import scipy.stats as scs
import copy
import openturns as ot
import timeit
from openturns.usecases import flood_model
import matplotlib.pyplot as plt


In [2]:
def cdf_emp(Y_ord,t):
    B = Y_ord <= t*np.ones(len(Y_ord))
    return np.sum(B)/len(Y_ord)

def quant_emp(Y_ord,alpha):
    cdf = [cdf_emp(Y_ord,t) for t in Y_ord]
    B = cdf <= alpha
    return np.sort(Y_ord)[np.sum(B) - 1]

In [116]:
print(scs.norm().ppf(0.95))
Z = scs.laplace().rvs(100)

1.6448536269514722
[ 0.47062424 -0.32246019 -0.02320026 -0.04592407  0.70575819  1.23519476
 -0.44462401  1.07115446  0.99414687 -0.79950472  1.11116224  0.72996523
  3.17212343 -0.54034596 -2.99883635  0.99032735  0.6438217  -1.93888536
  1.56108341  0.64594586 -2.24691172 -0.18849429  1.26034367 -7.20162668
 -0.77904454  0.39427701 -0.24606675 -0.85415838 -0.8735465  -0.4711789
  0.23679009  2.07742519 -4.39510774  2.35756006  0.45263767 -1.30218031
 -0.74994507 -0.3132239  -0.94837101 -0.37994388  0.37990991 -1.76334446
 -0.95429627 -1.76626938  0.99495875 -1.58565579  2.12540949 -0.62272913
 -2.67821288 -0.69614183  1.22806431 -0.36544283  0.26923537  0.33839695
 -0.50026411  1.01441483  0.19572106  1.38625664 -0.47829801 -0.15776222
 -2.16095937  0.54333919  0.64983956  0.42234024 -0.10577373 -0.63100419
 -1.61449126  1.35047055 -0.45800347 -0.13745419  0.40298915 -0.05432064
  0.58923684 -1.26123398 -1.80180012  1.93013172  1.42552094 -1.0587279
 -0.72789583 -1.05841948 -0.398421

In [114]:
N = 2000000
alpha = 0.95

Y = scs.norm().rvs(N)
Y_ord = np.sort(Y)
#t = np.linspace(-5,5,10)
#F = [cdf_emp(Y_ord,x) for x in t]
#plt.plot(t,F)

In [115]:
      #confidence level
#t = 0
#cdf_emp_t = cdf_emp(Y_ord,t)
E = [0.75,0.95,0.99]
for eps in E:
    eta = np.sqrt(-np.log((1-eps)/2)/(2*N))
    #print('Vraie valeur de F(t)='+str(scs.norm().cdf(t)))
    #print('DKW sur cdf : P(F(t) dans ['+str(cdf_emp_t - eta)+','+str(cdf_emp_t +eta)+'])>'+str(eps))
    print('IdCNA sur quantile : P( q_alpha dans ['+str(np.quantile(Y,alpha -eta))+','+str(np.quantile(Y,alpha +eta))+'] )>'+str(eps)) 

IdCNA sur quantile : P( q_alpha dans [1.6351787528677013,1.6493882764980234] )>0.75
IdCNA sur quantile : P( q_alpha dans [1.6328480036243227,1.6518061641069757] )>0.95
IdCNA sur quantile : P( q_alpha dans [1.6311604885912279,1.6537914939454266] )>0.99


In [ ]:
def L(th,th0,X):

    p_th = scs.norm(th[0],th[1]).pdf(X)
    p_th_0 = scs.norm(th0[0],th0[1]).pdf(X)
    
    return p_th/p_th_0

def cdf_emp_ris(X,Y_ord,N,th,th0,t):
    S = np.sum(L(th,th0,X))

    i = 0
    while t>=Y_ord[i] and i < N-1:
        i+=1
   
    return np.sum(L(th,th0,Y_ord[0:i]))/S  

sample = ot.Normal(0,1).getSample(200)
emp_cdf = [cdf_emp_ris(sample,[x]) for x in sample ]
print(empirical_cdf(sample,sample[2]))



In [25]:
from openturns.usecases import flood_model
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt

ot.Log.Show(ot.Log.NONE)

fm = flood_model.FloodModel()
distribution = fm.distribution
model = fm.model.getMarginal(0)
sample = distribution.getSample(10000)
sample

class=Sample name=ComposedDistribution implementation=class=SampleImplementation name=ComposedDistribution size=10000 dimension=8 description=[Q (m3/s),Ks,Zv (m),...,L (m),Zb (m),Hd (m)] data=[[932.621,29.2223,49.462,...,4997.93,55.6125,2.9079],[1997.1,38.7215,50.8709,...,5002.75,55.4795,3.68896],[1533.2,23.4371,49.4017,...,4998.96,55.5363,3.61339],...,[3407.22,28.9594,50.6112,...,4999.04,55.2002,3.50786],[3608.68,34.4764,49.9039,...,5001.15,55.719,3.76673],[443.801,28.0073,49.3829,...,5000.11,55.4005,2.4898]]

In [7]:
output = model(sample)
output

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=10000 dimension=1 description=[H] data=[[2.40264],[7.42544],[1.44039],...,[1.72518],[4.00529],[1.32595]]

In [ ]:
def ris_cdf_estimator(sample, grid, f, f_delta, g, dis_type="ot"):
    
    if dis_type == "ot":
        
        cdf = np.zeros((len(grid), len(sample)))
        likelihood_ratio = np.array([f_delta.computePDF(x) / f.computePDF(x) for x in sample])
        
        G = np.array([g, ] * len(grid))
        T = np.array([grid, ] * len(sample))
        B = G <= T.transpose()

        L = np.array([likelihood_ratio, ] * len(grid))
        B = L * B
        return np.sum(B, axis=1) / np.sum(likelihood_ratio)
    
    if dis_type == "custom":

        cdf = np.zeros((len(grid), len(sample)))
        likelihood_ratio = np.array([f_delta(x) / f.computePDF(x) for x in sample])

        G = np.array([g, ] * len(grid))
        T = np.array([grid, ] * len(sample))

        B = G <= T.transpose()

        L = np.array([likelihood_ratio, ] * len(grid))
        B = L * B

        return np.sum(B, axis=1) / np.sum(likelihood_ratio)

In [ ]:
model = fm.model.getMarginal(0)
sample = distribution.getSample(1000)
output = model(sample)
s = np.transpose(output)

print(output)
dist = ot.TruncatedDistribution(ot.Gumbel(558,1013), 500.0, 3000.0)
t = np.linspace(450,3050,10000)
L = np.array([dist.computePDF(x) for x in t])
plt.plot(t,L)
plt.show()
d = ot.TruncatedDistribution(ot.Normal(30,7.5), 15.0, 10.0**6)
l = np.linspace(10,60,100)
M = [d.computePDF(x) for x in l]
plt.plot(l,M)

In [ ]:
f = ot.Normal(30,7.1)
f_delta = ot.Normal(29,9)

t = np.linspace(-10,-5,100)
model = fm.model.getMarginal(0)
sample = distribution.getSample(100)
output = model(sample)

plt.plot(t,ris_cdf_estimator(sample[:,1],t,f,f_delta,np.transpose(output)[0],"ot"));
#print(ris_cdf_estimator(sample[:,1],t,f,f_delta,np.transpose(output)[0],"ot"))


In [ ]:
def quantile(alpha, cdf, grid):
    """
    Compute quantile of level alpha

    :param alpha: level of the quantile
    :param cdf: list of size T of the evaluation of cdf in grid
    :param grid: list of size T
    :return: float, the quantile of level alpha for the empirical CDF cdf
    """

    assert len(grid) == len(cdf)

    boolean = cdf <= alpha
    return grid[np.sum(boolean)]

In [ ]:
u = ris_cdf_estimator(sample[:,1],t,f,f_delta,np.transpose(output)[0])
print(quantile(0.1, u, t))

In [ ]:
sampleSize = 1000000
inputSample = distribution.getSample(sampleSize)
outputSample = model(inputSample)
graph = ot.HistogramFactory().build(outputSample).drawPDF()
_ = viewer.View(graph)

## We can create truncated distributions using openturns with the following code for the Gaussian family

In [ ]:
# truncation interval [a,b]

distribution = ot.TruncatedDistribution(ot.Normal(2.0, 1.5), 1.0, 2.0)

Distribution = ot.TruncatedDistribution(ot.Normal(), 1.0, 2.0)

f= ot.TruncatedDistribution(ot.Normal(),-1.0,-0.5)
theta=np.array([0,2,-1,-0.5])
f.setParameter(theta)
sample = f.getSample(10)
print(sample)
print(np.shape(sample))

## 1. Fisher information

In [26]:
def emp_fisher(args):
    """
    Helper function to compute loglikgrad * loglikgrad^T where loglikgrad is the gradient of the score function at point
    theta, evaluate in the point

    :param point: sample point where to evaluate
    :param f:
    :param theta:
    :return:
    """
    point, f, theta, d, dis_type = args
    f.setParameter(theta)
    x = np.array(f.computeLogPDFGradient([point]))
    y = []
    for i in range(d):
        y += [x[0][i]]
    y = np.array(y)
    y = y.reshape(d, 1)
    return np.matmul(y, np.transpose(y))


def fisher_information(f, theta, d, dis_type="ot"):
    """
    Compute the Fisher Information matrix at point theta for the distribution f

    :param f: Statistical distribution (openturns)
    :param theta: a list or numpy array of dimension d = number of parameters
    :param d: number of parameters
    :param dis_type: to precise if f is it a openturns distribution or a custom function

    :return: a (d, d) numpy array
    """

    # Get a large sample distributed by f(theta)

    N = 1000
    f.setParameter(theta)
    sample = f.getSample(N)

    # Compute gradient log likelihood
    pool = multiprocessing.Pool(100)
    args = [(point, f, theta, d, dis_type) for point in sample]
    fisher_info = pool.map(emp_fisher, args)
    fisher_info = np.mean(fisher_info, axis=0)

    # Fisher Information Matrix is always nonnegative, replacing negative values due to statistical fluctuation by
    # zeroes
    #return np.array([0 if fisher_info[i, j] < 0 else fisher_info[i, j] for i in range(d) for j in range(d)]).reshape(d,d)
    return fisher_info                                                                                                                


def fisher_ellipsoid(f, theta, delta, npoints, d):
    """
    Compute approximate sphere of radius delta w.r.t. the Fisher information at theta.

    :param f: distribution family
    :param theta: center point
    :param delta: radius of the sphere
    :param npoints: number of points to compute on the sphere
    :param d: number of parameters of the manifold
    :return: list of size npoints x 2
    """

    grid_t = np.linspace(0, 2 * np.pi, npoints)
    fi = fisher_information(f, theta, d)
    w, v = la.eig(fi)
    new_theta = np.matmul(np.transpose(v), np.array(theta[:d]))
    ellipsoid_eigbasis = [
        np.array([new_theta[0] + np.cos(t) / np.sqrt(w[0]) * delta, new_theta[1] + np.sin(t) / np.sqrt(w[1]) * delta])
        for t in grid_t]
    ellipsoid = [np.matmul(v, x) for x in ellipsoid_eigbasis]
    return ellipsoid

In [27]:
p = [10,1]
#p = [0,1]
f=ot.Gumbel()
start = timeit.default_timer()
fi = fisher_information(f,p,2)
stop = timeit.default_timer()
print(stop - start,"secondes")

print(fi)
print("determinant =",np.linalg.det(fi))

3.8427322100033052 secondes
[[ 0.01767133 -0.00417475]
 [-0.00417475  0.00993266]]
determinant = 0.00015809476477050415


In [12]:
p = np.array([0,1])
f = ot.Normal(0,1)
start = timeit.default_timer()
fi = fisher_information(f,p,2)
stop = timeit.default_timer()
print(stop - start,"secondes")
print(fi)
for i in range(4):
    print(fisher_information(f,p,2))

3.9837221929992666 secondes
[[0.97260679 0.00225082]
 [0.00225082 1.81044781]]
[[ 1.07117532 -0.15028478]
 [-0.15028478  2.63629734]]
[[0.96714192 0.10493808]
 [0.10493808 1.964662  ]]
[[0.99326907 0.02816632]
 [0.02816632 2.13939359]]
[[ 0.95791812 -0.04844706]
 [-0.04844706  1.85576053]]


In [13]:
f = ot.Normal(0,1)
sample = f.getSample(5)
print(sample)

#fittedRes = ot.NormalFactory().buildEstimator(sample)
#paramDist = fittedRes.getParameterDistribution()
#paramDist

    [ X0         ]
0 : [ -1.66773   ]
1 : [  0.655403  ]
2 : [ -0.309728  ]
3 : [  0.173697  ]
4 : [ -0.0998528 ]


## 2. Fisher Triangular

In [4]:
def fisher_triangular_inv(c, a, b):
    grid = np.linspace(a, b, 100)
    f = ot.Triangular(a, c, b)
    val = [f.computeLogPDFGradient([t])[1] ** 2 * f.computePDF([t]) for t in grid]
    fi = spi.simps(val, grid)
    return 1 / fi

print(fisher_triangular_inv(2,1,5))

NameError: name 'np' is not defined

## 3. Geodesic solver

In [14]:
# This is the solver used to compute Fisher sphere for parametric models with 2 parameters only (e.g. the Gaussian laws)


def fi_inv(distribution, theta, type="bounded", ngrid=100):
    """
    Helper function to compute an approximation of the inverse Fisher information matrix (FIM) of f in theta,
    based on Simpson's method

    :param distribution: Openturns distribution object
    :param theta: tuple of parameters where to evaluate the Inverse Fisher Information matrix,
                  (if bounded, bounds value needs to be added after the parameters)
    :param type: string, can be "bounded", "left truncated" or "unbounded", This is needed for defining the interval of
                 Simpson's quadrature method
    :return: 2x2 numpy array, The inverse FIM
    """

    f = copy.deepcopy(distribution)

    if type == "bounded":
        a, b = theta[-2], theta[-1]
        q1, q2 = theta[0], theta[1]
        grid = np.linspace(a, b, ngrid)
        f.setParameter((q1, q2, a, b))
    if type == "left truncated":
        a = theta[-1]
        q1, q2 = theta[0], theta[1]
        f.setParameter((q1, q2, a))
        grid = np.linspace(a, a + 10 * f.getStandardDeviation()[0], ngrid)
        f.setParameter((q1, q2, a))

    if type == "unbounded":
        q1, q2 = theta[0], theta[1]
        f.setParameter((q1, q2))
        grid = np.linspace(f.getMean()[0] - 6 * f.getStandardDeviation()[0], f.getMean()[0] + 6 * f.getStandardDeviation()[0], ngrid)
        f.setParameter((q1, q2))

    val = []
    for i in range(2):
        for j in range(2):
            val += [
                [f.computeLogPDFGradient([t])[i] * f.computeLogPDFGradient([t])[j] * f.computePDF(t) for t in grid]]

    for k in range(len(val)):
        val[k] = spi.simps(val[k], grid)
    fi = np.array(val).reshape(2, 2)
    return la.inv(fi)


def sqr(inv_f):
    """
    Helper function to compute the square root of a symmetric definite positive matrix.

    :param inv_f: the matrix to compute the square root
    :return:  2x2 numpy array
    """
    w, v = la.eig(inv_f)
    sqr_inv_f = np.dot(np.dot(v, np.diag(np.sqrt(w))), np.transpose(v))
    return sqr_inv_f


def grad_fi_inv(inv_f, distribution, theta, type="bounded", ngrid=100):
    """
    Helper function to compute the partial derivative of the Inverse FIM wrt to each parameter (with the matrix square root
    method).

    :param inv_f: inverse FIM in theta
    :param f: Openturns distribution object
    :param theta: tuple of parameters where to evaluate the gradient (if bounded, bounds value needs to be added after the parameters)
    :return: a tuple of 2x2 numpy array
    """
    
    f = copy.deepcopy(distribution)

    s = sqr(inv_f)
    h1 = np.dot(s, np.array([10 ** (-3), 0]).reshape(2, 1))
    h2 = np.dot(s, np.array([0, 10 ** (-3)]).reshape(2, 1))
    h = np.array([h1, h2]).reshape(2, 2)
    if type == "bounded":
        a, b = theta[-2], theta[-1]
        q1, q2 = theta[0], theta[1]
        f2 = fi_inv(f, (q1 + h1[0][0], q2 + h1[1][0], a, b), ngrid=ngrid) - inv_f
        f3 = fi_inv(f, (q1 + h2[0][0], q2 + h2[1][0], a, b), ngrid=ngrid) - inv_f
    if type == "left truncated":
        a = theta[-1]
        q1, q2 = theta[0], theta[1]
        f2 = fi_inv(f, (q1 + h1[0][0], q2 + h1[1][0], a), type="left truncated", ngrid=ngrid) - inv_f
        f3 = fi_inv(f, (q1 + h2[0][0], q2 + h2[1][0], a), type="left truncated", ngrid=ngrid) - inv_f
    if type == "unbounded":
        q1, q2 = theta[0], theta[1]
        f2 = fi_inv(f, (q1 + h1[0][0], q2 + h1[1][0]), type="unbounded", ngrid=ngrid) - inv_f
        f3 = fi_inv(f, (q1 + h2[0][0], q2 + h2[1][0]), type="unbounded", ngrid=ngrid) - inv_f

    gfi0 = np.zeros((2, 2))
    gfi1 = np.zeros((2, 2))

    for i in range(2):
        for j in range(2):
            g = np.dot(la.inv(h), np.array([f2[i, j], f3[i, j]]).reshape(2, 1))
            gfi0[i, j] = g[0]
            gfi1[i, j] = g[1]
    return gfi0, gfi1


def grad_fi_inv2(inv_f, distribution, theta):
    """
    Helper function to compute the partial derivative of the Inverse FIM wrt to each parameter. (with the optimisation based method)

    :param inv_f: inverse FIM in theta
    :param distribution: Openturns distribution object
    :param theta: Parameter where to evaluate the gradient
    :return: a tuple of 2x2 numpy array
    """

    f = copy.deepcopy(distribution)
    a, b = theta[-2], theta[-1]
    q1, q2 = theta[0], theta[1]
    eps = 10**(-5)
    e1 = np.array([1, 0])
    e2 = np.array([0, 1])
    h1 = np.dot(inv_f, e1.reshape(2, 1))
    h1 = h1 / np.sqrt(np.dot(e1.T, np.dot(inv_f, e1)))
    h2 = np.dot(inv_f, e2.reshape(2, 1))
    h2 = h2 / np.sqrt(np.dot(e2.T, np.dot(inv_f, e2)))
    h = np.array([h1, h2]).reshape(2, 2)
    f2 = fi_inv(f, (q1 + eps * h1[0][0], q2 + eps * h1[1][0], a, b)) - inv_f
    f3 = fi_inv(f, (q1 + eps * h2[0][0], q2 + eps * h2[1][0], a, b)) - inv_f
    gfi0 = np.zeros((2, 2))
    gfi1 = np.zeros((2, 2))

    for i in range(2):
        for j in range(2):
            g = np.dot(la.inv(h), np.array([f2[i, j], f3[i, j]]).reshape(2, 1))
            gfi0[i, j] = g[0]
            gfi1[i, j] = g[1]
    return gfi0, gfi1


def hamiltonian(y, t, distribution, theta, type="bounded", ngrid=100):
    """
    Function to pass as argument to scipy.integrate.odeint to compute the geodesic

    :param y: (q1, q2, p1, p2) coordinate of the mobile in the manifold, q1 and q2 match the paramaters,
                p1 and p2 to the "speed" of the particule
    :param t: time argument, mandatory for odeint
    :param f: Openturns distribution object
    :param theta: coordinate + the max and min of the distribution support if type == "bounded"
    :return: the value of the time derivative of y
    """
    
    f = copy.deepcopy(distribution)
    params = theta[0], theta[1]
    q1, q2, p1, p2 = y
    if type == "bounded":
        inv_f = fi_inv(f, (q1, q2, theta[-2], theta[-1]), type=type, ngrid=ngrid)
        # inv_f = la.inv(fisher_information(f, (q1, q2, 0, theta[2], theta[3]), 2))
        gfi = grad_fi_inv(inv_f, f, (q1, q2, theta[-2], theta[-1]), ngrid=ngrid)
        f.setParameter(theta)
    if type == "left truncated":
        inv_f = fi_inv(f, (q1, q2, theta[-1]), type=type, ngrid=ngrid)
        gfi = grad_fi_inv(inv_f, f, (q1, q2, theta[-1]), type=type, ngrid=ngrid)
    if type == "unbounded":
        inv_f = fi_inv(f, (q1, q2), type=type, ngrid=ngrid)
        gfi = grad_fi_inv(inv_f, f, (q1, q2), type=type, ngrid=ngrid)

    dq = np.dot(inv_f, np.array([p1, p2]))
    dp = [-0.5 * np.dot(np.array([p1, p2]).T, np.dot(gfi[0], np.array([p1, p2]))),
          -0.5 * np.dot(np.array([p1, p2]).T, np.dot(gfi[1], np.array([p1, p2])))]
    return np.array([dq[0], dq[1], dp[0], dp[1]])


def hamiltonian_val(y, distribution, theta, type="unbounded"):
    """
    Compute value of the hamiltonian in y
    :param y: parameter
    :param theta: tuple of parameters
    :param f: Openturns ditribution object
    :return: float value, Hamiltonian value
    """
    f = copy.deepcopy(distribution)
    if type == "unbounded":
        q1, q2, p1, p2 = y
        f.setParameter((q1, q2))
        inv_f = fi_inv(f, (q1, q2), type="unbounded")
        f.setParameter((theta[0], theta[1]))
        return 0.5 * np.dot(np.array([p1, p2]).T, np.dot(inv_f, np.array([p1, p2])))
    if type == "bounded":
        q1, q2, p1, p2 = y
        a, b = theta[2], theta[3]
        f.setParameter((q1, q2, a, b))
        inv_f = fi_inv(f, (q1, q2, a, b), type="bounded")
        f.setParameter(theta)
        return 0.5 * np.dot(np.array([p1, p2]).T, np.dot(inv_f, np.array([p1, p2])))
    if type == "normal":
        q1, q2, p1, p2 = y
        inv_f = np.array([[q2**2, 0], [0, q2**2/2]]).reshape(2, 2)
        return 0.5 * np.dot(np.array([p1, p2]).T, np.dot(inv_f, np.array([p1, p2])))


def hamiltonian_moeu(y, t, theta):
    """
    Compute value of the hamiltonian in y for the Marshall Olkin Extended Uniform family
    :param y: parameter
    :param theta: tuple of parameters
    :param f: Openturns ditribution object
    :return: float value, Hamiltonian value
    """
    q, p = y
    inv_f = fisher_moeu_inv(q, theta[1], theta[2])
    gfi = (fisher_moeu_inv(q + 10 ** (-3) * np.sqrt(inv_f), theta[1], theta[2]) - inv_f) / (10 ** (-3) * np.sqrt(inv_f))
    dq = inv_f * p
    dp = -0.5 * p ** 2 * gfi
    return np.array([dq, dp])


def hamiltonian_triangular(y, t, theta):
    """
    Compute value of the hamiltonian in y for the Triangular family
    :param y: parameter
    :param theta: tuple of parameters
    :param f: Openturns ditribution object
    :return: float value, Hamiltonian value
    """
    q, p = y
    inv_f = fisher_triangular_inv(q, theta[1], theta[2])
    gfi = (fisher_triangular_inv(q + 10 ** (-3) * np.sqrt(inv_f), theta[1], theta[2]) - inv_f) / (10 ** (-3) * np.sqrt(inv_f))
    dq = inv_f * p
    dp = -0.5 * p ** 2 * gfi
    return np.array([dq, dp])


def explicit_euler(fun, y0, t):
    """
    Explicit Euler implementation
    :param fun: function such that y' = fun(y, t)
    :param y0: initial condition
    :param t: time grid
    :return: array of the solution at each time step
    """
    h = float((t[-1] - t[0])/len(t))
    sol = [y0 + h*fun(t[0], y0)]

    for i in range(len(t) - 1):
        sol += [sol[-1] + h*fun(t[i], sol[-1])]

    return np.array(sol)



In [31]:
f = ot.TruncatedDistribution(ot.Gumbel(10.,1.),1.,2.)
y0 = [10.,1.,1.,1.]
t = np.linspace(0,5,20)
theta = [0,1,1.,100.]
C=explicit_euler(lambda t, y: hamiltonian(y, t, f, theta, type="bounded"), y0, t)
plt.figure(figsize=(8,8))
plt.plot(C[:,0],C[:,1])
plt.scatter(y0[0],y0[1])
plt.xlim(-1,2)
plt.ylim(0,3)

/tmp/ipykernel_7244/1900662101.py:44: DeprecationWarning: 'scipy.integrate.simps' is deprecated in favour of 'scipy.integrate.simpson' and will be removed in SciPy 1.14.0
  val[k] = spi.simps(val[k], grid)
/tmp/ipykernel_7244/1900662101.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gfi0[i, j] = g[0]
/tmp/ipykernel_7244/1900662101.py:100: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gfi1[i, j] = g[1]


TypeError: InvalidArgumentException : Beta MUST be positive

In [ ]:

def compute_fisher_sphere(args):
    """
    Helper for computing multiple geodesics in parallel
    :param args: (f, y0, theta)
    :return: the coordinate of the Fisher sphere
    """

    f, y0, theta, type = args
    t = np.linspace(0, 1, 100)
    sol = explicit_euler(lambda t, y: hamiltonian(y, t, f, theta, type=type), y0, t)
    return [sol[:, 0][-1], sol[:, 1][-1]]


def compute_fisher_sphere_moeu(args):
    """
    Helper for computing multiple geodesics in parallel
    :param args: (f, y0, theta)
    :return: the coordinate of the Fisher sphere
    """

    f, y0, theta, type = args
    t = np.linspace(0, 1, 100)
    try:
        sol = explicit_euler(lambda t, y: hamiltonian_moeu(y, t, f, theta), y0, t)
        if type == "end":
            return [sol[:, 0][-1]]
        if type == "all":
            return [sol[:, 0]]
    except:
        return [0, 0]


def compute_fisher_sphere_triangular(args):
    """
    Helper for computing multiple geodesics in parallel
    :param args: (f, y0, theta)
    :return: the coordinate of the Fisher sphere
    """
    f, y0, theta = args
    t = np.linspace(0, 1, 100)
    sol = explicit_euler(lambda t, y: hamiltonian_triangular(y, t, f, theta), y0, t)
    return [sol[:, 0][-1]]

In [28]:
def I(theta):
    m = theta[0]
    s = theta[1]
    return 1/s**2 * np.array([[1,0],[0,2]])

def sample_Fisher_sphere(theta_0,f,delta,nb_pts):
    Liste_pts = np.linspace(0,2*np.pi,nb_pts)

    C = np.zeros((nb_pts,2))    # Va contenir la liste des points de la sphère approximée
    t = np.linspace(0, 1, 100)
    
    j = 0
    for t in Liste_pts:
        #Ici on prend un vecteur v d'angle t et on le renormalise pour que |v| = delta
        v = np.array([np.cos(t),np.sin(t)]) 
        l = np.sqrt(np.dot(np.transpose(v),np.dot(I(theta_0),v)))
        v = delta*(v/l)

        # Ici on lance le calcul de géodésique en partant de theta_0 avec vecteur vitesse initiale v
        y0 = [theta_0[0],theta_0[1],v[0],v[1]]
        args = f,y0,theta_0,"unbounded"
        sol = explicit_euler(lambda t, y: hamiltonian(y, t, f, theta_0, type="unbounded"), y0, t)
        
        C[j,:] = np.array([sol[:, 0][-1], sol[:, 1][-1]])                      # on stocke le point final de la géodésique dans C
        j+=1
    return C

a=-1
b=1
theta_0 = [0,1]
f = ot.Normal()
delta = 0.5
nb_pts = 60

start = timeit.default_timer()
C = sample_Fisher_sphere(theta_0,f,delta,nb_pts)
stop = timeit.default_timer()
print(stop - start,"secondes")
plt.scatter(theta_0[0],theta_0[1])
plt.plot(C[:,0],C[:,1])
plt.xlim(-0.75,0.75)
plt.ylim(0.25,1.75);

IndexError: invalid index to scalar variable.

In [125]:
1-ot.ChiSquare(2.).computeCDF(50)
#print(10/np.sqrt(1000))

1.3887890837338546e-11

In [ ]:

def symplectic_scheme(y0, t, distribution, omega, type="bounded", bounds=(0,0)):

    """
    Symplectic scheme for non separable hamiltonian (inspired from https://arxiv.org/pdf/1609.02212.pdf)
    :param y0: initial condition
    :param t: time range array in [0, 1]
    :param f: Openturns distribution object
    :param theta: tuple of initial parameters
    :param omega: bounding factor
    :param type: "bounded", "unbounded" or "left truncated"
    :return:
    """
    f = copy.deepcopy(distribution)
    h = float((t[-1] - t[0]) / len(t))
    q1, q2, p1, p2, x1, x2, y1, y2 = y0
    q1, q2 = y0[0], y0[1]
    x1, x2 = y0[4], y0[5]
    if type == "unbounded":
        q = np.array([q1, q2])
        p = np.array([p1, p2])
        x = np.array([x1, x2])
        y = np.array([y1, y2])

        # step 1: time flow of H_A
        f.setParameter((q1, q2))
        inv_f = fi_inv(f, (q1, q2), type=type)
        gfi = grad_fi_inv(inv_f, f, (q1, q2), type=type)

        dp = np.array([0.5 * np.dot(y.T, np.dot(gfi[0], y)), 0.5 * np.dot(y.T, np.dot(gfi[1], y))])
        p = p - h / 2 * dp

        dx = np.dot(inv_f, y)
        x = x + h / 2 * dx

        # step 2: time flow of H_B

        f.setParameter((x1, x2))
        inv_f = fi_inv(f, (x1, x2), type=type)
        gfi = grad_fi_inv(inv_f, f, (x1, x2), type=type)

        dq = np.dot(inv_f, p)
        q = q + h / 2 * dq

        dy = np.array([0.5 * np.dot(p.T, np.dot(gfi[0], p)), 0.5 * np.dot(p.T, np.dot(gfi[1], p))])
        y = y - h / 2 * dy

        # step 3: time flow of omega * H_C

        A = np.cos(2 * omega * h) * np.eye(2)
        B = np.sin(2 * omega * h) * np.eye(2)

        R = np.block([
            [A, B],
            [-B, A]
        ])

        mid = np.block([(q + x)/2, (p + y)/2]).reshape(4, 1)
        dif = np.block([(q - x)/2, (p - y)/2]).reshape(4, 1)
        new1 = mid + np.dot(R, dif)
        new2 = mid - np.dot(R, dif)

        q1, q2, p1, p2 = new1[:, 0]
        x1, x2, y1, y2 = new2[:, 0]

        q = np.array([q1, q2])
        p = np.array([p1, p2])
        x = np.array([x1, x2])
        y = np.array([y1, y2])

        # step 4: time flow of H_B

        f.setParameter((x1, x2))
        inv_f = fi_inv(f, (x1, x2), type=type)
        gfi = grad_fi_inv(inv_f, f, (x1, x2), type=type)

        dq = np.dot(inv_f, p)
        q = q + h / 2 * dq

        dy = np.array([0.5 * np.dot(p.T, np.dot(gfi[0], p)), 0.5 * np.dot(p.T, np.dot(gfi[1], p))])
        y = y - h / 2 * dy

        # step 5: time flow of H_A
        f.setParameter((q1, q2))
        inv_f = fi_inv(f, (q1, q2), type=type)
        gfi = grad_fi_inv(inv_f, f, (q1, q2), type=type)

        dp = np.array([0.5 * np.dot(y.T, np.dot(gfi[0], y)), 0.5 * np.dot(y.T, np.dot(gfi[1], y))])
        p = p - h / 2 * dp

        dx = np.dot(inv_f, y)
        x = x + h / 2 * dx

        q1, q2 = q
        y1, y2 = y
        x1, x2 = x
        p1, p2 = p

    if type == "bounded":
        a, b = bounds
        q = np.array([q1, q2])
        p = np.array([p1, p2])
        x = np.array([x1, x2])
        y = np.array([y1, y2])

        # step 1: time flow of H_A
        f.setParameter((q1, q2, a, b))
        inv_f = fi_inv(f, (q1, q2, a, b), type=type)
        gfi = grad_fi_inv(inv_f, f, (q1, q2, a, b), type=type)

        dp = np.array([0.5 * np.dot(y.T, np.dot(gfi[0], y)), 0.5 * np.dot(y.T, np.dot(gfi[1], y))])
        p = p - h / 2 * dp

        dx = np.dot(inv_f, y)
        x = x + h / 2 * dx

        # step 2: time flow of H_B

        f.setParameter((x1, x2, a, b))
        inv_f = fi_inv(f, (x1, x2, a, b), type=type)
        gfi = grad_fi_inv(inv_f, f, (x1, x2, a, b), type=type)

        dq = np.dot(inv_f, p)
        q = q + h / 2 * dq

        dy = np.array([0.5 * np.dot(p.T, np.dot(gfi[0], p)), 0.5 * np.dot(p.T, np.dot(gfi[1], p))])
        y = y - h / 2 * dy

        # step 3: time flow of omega * H_C

        A = np.cos(2 * omega * h) * np.eye(2)
        B = np.sin(2 * omega * h) * np.eye(2)

        R = np.block([
            [A, B],
            [-B, A]
        ])

        mid = np.block([(q + x) / 2, (p + y) / 2]).reshape(4, 1)
        dif = np.block([(q - x) / 2, (p - y) / 2]).reshape(4, 1)
        new1 = mid + np.dot(R, dif)
        new2 = mid - np.dot(R, dif)

        q1, q2, p1, p2 = new1[:, 0]
        x1, x2, y1, y2 = new2[:, 0]

        q = np.array([q1, q2])
        p = np.array([p1, p2])
        x = np.array([x1, x2])
        y = np.array([y1, y2])

        # step 4: time flow of H_B

        f.setParameter((x1, x2, a, b))
        inv_f = fi_inv(f, (x1, x2, a, b), type=type)
        gfi = grad_fi_inv(inv_f, f, (x1, x2, a, b), type=type)

        dq = np.dot(inv_f, p)
        q = q + h / 2 * dq

        dy = np.array([0.5 * np.dot(p.T, np.dot(gfi[0], p)), 0.5 * np.dot(p.T, np.dot(gfi[1], p))])
        y = y - h / 2 * dy

        # step 5: time flow of H_A
        f.setParameter((q1, q2, a, b))
        inv_f = fi_inv(f, (q1, q2, a, b), type=type)
        gfi = grad_fi_inv(inv_f, f, (q1, q2, a, b), type=type)

        dp = np.array([0.5 * np.dot(y.T, np.dot(gfi[0], y)), 0.5 * np.dot(y.T, np.dot(gfi[1], y))])
        p = p - h / 2 * dp

        dx = np.dot(inv_f, y)
        x = x + h / 2 * dx

        q1, q2 = q
        y1, y2 = y
        x1, x2 = x
        p1, p2 = p

    return q1, q2, p1, p2, x1, x2, y1, y2


def symplectic_second_order(y0, t, f, omega, type="bounded", bounds=(0,0)):

    sol = [y0]
    params = f.getParameter()
    for i in range(len(t) - 1):
        f.setParameter(params)
        sol += [symplectic_scheme(sol[-1], t, f, omega, type=type, bounds=bounds)]

    sol = np.array(sol)
    sol.reshape(len(t), 8)
    f.setParameter(params)

    return sol



def initial_speed(inv_f, delta, npoints):
    """
    Helper function to compute the initial speed vectors necessary to throw our particule at distance delta for t = 1

    :param inv_f: the inverse FIM
    :param delta: the Fisher sphere radius
    :param npoints: number of vectors to compute
    :return:
    """
    grid_t = np.linspace(0, 2 * np.pi, npoints)
    w, v = la.eig(inv_f)
    ellipsoid_eigbasis = [
        np.array([np.cos(t) / np.sqrt(w[0]) * delta, np.sin(t) / np.sqrt(w[1]) * delta])
        for t in grid_t]
    ellipsoid = [np.matmul(v, x) for x in ellipsoid_eigbasis]
    return ellipsoid


In [18]:
def moeu(alpha, a, b, x):
    return alpha * (b - a) / (alpha * b - a + (1 - alpha) * x) ** 2


def fisher_moeu_inv(alpha, a, b):
    sample = np.linspace(a, b, 100)
    val = np.array(
        [((np.log(moeu(alpha + 0.01, a, b, x)) - np.log(moeu(alpha, a, b, x)))/0.01) ** 2 * moeu(alpha, a, b, x) for x in
         sample])
    fi = spi.simps(val, sample)
    return 1/fi


class Moeu:

    def __init__(self, alpha, a, b):
        self.alpha = alpha
        self.a = a
        self.b = b

    def computePDF(self, x):
        return moeu(self.alpha, self.a, self.b, x)

## 4. PLI

In [19]:
def empirical_cdf(sample, grid):

    cdf = np.zeros((len(grid), len(sample)))

    G = np.array([sample, ] * len(grid))
    T = np.array([grid, ] * len(sample))

    B = G <= T.transpose()

    return np.sum(B, axis=1)/len(sample)


def ris_cdf_estimator(sample, grid, f, f_delta, g, dis_type="ot"):
    """
    RIS (Reverse Importance Sampling) estimator of the CDF of g with distribution f_delta

    :param sample: list of size N of values sampled with f
    :param grid: point where to evaluate CDF
    :param f: original distribution
    :param f_delta: perturbated distribution
    :param g: list of size N of the evaluation of the numerical model with sample
    :param dis_type: "ot" if f_delta is a Openturns distribution, "custom" if f_delta custom function
    :return: float value, CDF RIS estimator in t
    """


    if dis_type == "ot":

        cdf = np.zeros((len(grid), len(sample)))
        likelihood_ratio = np.array([f_delta.computePDF(x) / f.computePDF(x) for x in sample])

        G = np.array([g, ] * len(grid))
        T = np.array([grid, ] * len(sample))

        B = G <= T.transpose()

        L = np.array([likelihood_ratio, ] * len(grid))
        B = L * B

        return np.sum(B, axis=1) / np.sum(likelihood_ratio)



    if dis_type == "custom":

        cdf = np.zeros((len(grid), len(sample)))
        likelihood_ratio = np.array([f_delta(x) / f.computePDF(x) for x in sample])

        G = np.array([g, ] * len(grid))
        T = np.array([grid, ] * len(sample))

        B = G <= T.transpose()

        L = np.array([likelihood_ratio, ] * len(grid))
        B = L * B

        return np.sum(B, axis=1) / np.sum(likelihood_ratio)



def quantile(alpha, cdf, grid):
    """
    Compute quantile of level alpha

    :param alpha: level of the quantile
    :param cdf: list of size T of the evaluation of cdf in grid
    :param grid: list of size T
    :return: float, the quantile of level alpha for the empirical CDF cdf
    """

    assert len(grid) == len(cdf)

    boolean = cdf <= alpha
    return grid[np.sum(boolean)]


def rosenblatt_cdf_estimator(sample, t, f, delta, g):
    """
    RIS (Reverse Importance Sampling) estimator of the CDF of g with distribution f_delta

    :param sample: list of size N of values sampled with f
    :param t: point where to evaluate CDF
    :param f: original distribution
    :param f_delta: perturbated distribution
    :param g: list of size N of the evaluation of the numerical model with sample
    :return: float value, CDF RIS estimator in t
    """
    i = 0
    denominator = np.sum([np.exp((2*delta*ot.Normal().computeQuantile(f.computeCDF(x))[0] - delta**2)/2) for x in sample])
    j = 0
    for x in sample:
        if g[j] <= t:
            i += np.exp((2*delta*ot.Normal().computeQuantile(f.computeCDF(x))[0] - delta**2)/2)
        j += 1
    return i / denominator


def pli(q, q_delta):
    """
    Compute the PLI

    :param q: alpha quantile of the original distribution
    :param q_delta: alpha quantile of the perturbated distribution
    :return: float, value of the PLI
    """
    return q_delta / q - 1
    


def compute_pli(args):
    """
    Helper function to compute the PLI with the multiprocessing library

    :param args: 6 - uplet corresponding of the f sample, the original quantile, f, f_delta and the numerical code g
    :return: float, pli value
    """
    sample, q, grid, f, f_delta, g, dis_type = args
    if dis_type == "ot":
        q_delta = quantile(0.95,
                           [ris_cdf_estimator(sample, t, f, f_delta, g) for
                            t in grid],
                           grid)
        return pli(q, q_delta)

    if dis_type == "custom":
        cdf = [ris_cdf_estimator(sample, t, f, f_delta, g, "custom") for
                            t in grid]
        q_delta = quantile(0.95,
                           cdf,
                           grid)
        del cdf
        return pli(q, q_delta)

In [42]:
def h(x):
     return x**3

def t(x,N):
    return np.sum(x)/N

N = 1000
M = 10000 # bootstrap samples

X = scs.norm(0,1).rvs(N)    

start = timeit.default_timer()
delta_diff = []

#the initial value of the statistic before bootstrap
t_0 = t(h(X),N)

for i in range(M):
    
    # generate a uniform integer sample on [0,N] of size N
    U = np.random.randint(N, size=N)

    # define the bootstrap sample
    H_star = h(X[U])

    #compute the statistic on the bootstrap sample
    t_star_i = t(H_star,N)

    #append the difference to a list
    delta_diff.append(t_star_i - t_0)
    
l_bound,u_bound = t_0 - np.sort(delta_diff)[int(0.9*M)], t_0 - np.sort(delta_diff)[int(0.1*M)]
Conf_interval = [l_bound,u_bound]
stop = timeit.default_timer()
print(stop - start,"seconds")
print(np.sort(delta_diff)[int(0.975*M)],np.sort(delta_diff)[int(0.025*M)])
print(Conf_interval)
print(t_0)

0.7109100720845163 seconds
0.236380254794726 -0.22457153706561603
[-0.014862534233578806, 0.28368492998376543]
0.1350036353083842


In [33]:
#print(np.sort(delta_diff))


[0.8996729779324075, 1.0075601575446265]
0.9535602634648969


In [34]:
#def my_stat(x,N):
#    return t(h(x),N)

#CI = scs.bootstrap(X,my_stat)

In [3]:
import utils.truncated_Gaussians as trGauss
import utils.truncated_Gumbel as trGumb
import utils.truncated_Triangular as trTriang

In [35]:
def quantile_estim_fast(F_is,Sample_input,H_val,theta_0,theta,alpha,a,b):
    # order the H_val list
    H_val_ord = np.sort(H_val)
    
    i = int(alpha*len(H_val))

    if F_is(Sample_input,H_val,theta_0,theta,[H_val_ord[i]],a,b) < alpha:
        while F_is(Sample_input,H_val,theta_0,theta,[H_val_ord[i]],a,b) < alpha:
            i = i+1
        return H_val_ord[i-1] # the -1 is because of python's indexing convention

    else:
        while F_is(Sample_input,H_val,theta_0,theta,[H_val_ord[i]],a,b)>= alpha:
            i = i-1
        return H_val_ord[i] #again we use i instead of i-1 because of python's indexing convention

In [38]:
N = 1000
alpha = 0.8
def HH(x):
    return x**3-3*x

th0_Gauss = [2,3]
sample_K = scs.norm(th0_Gauss[0],th0_Gauss[1]).rvs(N)
H_val = HH(sample_K)

theta = [2,3.1]
a_K,b_K = -2,3

start = timeit.default_timer()
print("quantile current method=",quantile_estim_fast(trGauss.F_is,sample_K,H_val,th0_Gauss,theta,alpha,a_K,b_K))
stop = timeit.default_timer()
print(stop - start,"seconds")
print("")

start = timeit.default_timer()
print("quantile with initial method=",trGauss.quant_estim(sample_K,H_val,th0_Gauss,theta,alpha,a_K,b_K))
stop = timeit.default_timer()
print(stop - start,"seconds")
print("")

quantile current method= 78.88629865033872
0.036823333939537406 seconds

quantile with initial method= 78.88629865033872
3.1168442579219118 seconds

